In [ ]:
# SQL
from sqlalchemy import create_engine, text
from contextlib import contextmanager
from sqlalchemy.orm import sessionmaker

# DATOS
import pandas as pd

In [2]:
class Datos:
    ### SQL & ML
    def __init__(self):
        self.hostname = "localhost"
        self.dbname = "crimewarehouse"
        self.uname = "root"
        self.pwd = "admin1234"

    def engine_mysql(self):
        #modules["cx_Oracle"] = oracledb
        engine = create_engine(
            'mysql+pymysql://',
            connect_args = {
                'user': self.uname,
                'password': self.pwd, 
                'host': self.hostname,
                'db':self.dbname},
            isolation_level="READ COMMITTED"
            )
        return engine
        
    @contextmanager
    def db_session(self):
        engine = self.engine_mysql()
        Session = sessionmaker(bind = engine)
        session = Session()
        try:
            print("🟢 Sesión de base de datos establecida...")
            yield session
        except Exception as e:
            session.rollback() 
            raise e
        finally:
            print("🔴 Cerrando sesión de base de datos...")
            session.close() 
    
    def get_contextual_crimes(self):
        query = text("""
                SELECT
                TIPO_DELITO_DESC
                FROM
                DIM_TIPO_DELITO""")
        
        with self.db_session() as connection:
            result = connection.execute(query)
            delitos = result.fetchall()
            
        lista_delitos = pd.DataFrame(delitos)
        lista_delitos = lista_delitos['TIPO_DELITO_DESC'].unique()
        
        return lista_delitos

In [3]:
clase_datos = Datos()

In [4]:
lista_delitos = clase_datos.get_contextual_crimes()

🟢 Sesión de base de datos establecida...
🔴 Cerrando sesión de base de datos...


In [5]:
lista_delitos

array(['Robo', 'Robo automotor', 'Hurto', 'Hurto automotor', 'Amenazas',
       'Lesiones Dolosas', 'Homicidios dolosos',
       'Lesiones en siniestros viales'], dtype=object)